<a href="https://colab.research.google.com/github/JoseAguirre-10/Tarea-1-Optimizaci-n-Matem-tica/blob/main/P1_Continua.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install gurobipy

In [15]:
import numpy as np #importamos la libreria de numpy para manejar matrices.
import gurobipy as gp #importamos la libreria de gurobi para optimizar el problema.
import math

In [16]:
#definamos la matriz D de la siguiente manera:

D = np.array([[4,6,5,9,17,4],[8,12,3,7,11,5],[6,8,2,10,15,7],[5,9,4,8,13,6],
 [7,11,3,12,8,10],[10,5,6,4,9,7]])
print(D)
np.shape(D) #para comprobar la dimensión de la matriz

def f(d):
    return 50000 * math.tanh(d - 5) + 50000

# Matriz de costos C aplicando f a cada elemento de D
C = np.vectorize(f)(D)


print(D[0,0])

[[ 4  6  5  9 17  4]
 [ 8 12  3  7 11  5]
 [ 6  8  2 10 15  7]
 [ 5  9  4  8 13  6]
 [ 7 11  3 12  8 10]
 [10  5  6  4  9  7]]
4


In [17]:
#primero voy a sacar los esfuerzos que requiere cada robot para mover cada objeto:
R1 = D[:,0]
R2 = D[:,1]
R3 = D[:,2]
R4 = D[:,3]
R5 = D[:,4]
R6 = D[:,5]

#ahora colocare todas las combinaciones posibles para cada robot donde cada robot se le asigna un solo objeto

movimiento = [np.array([R1[a], R2[b], R3[c], R4[d], R5[e], R6[f]]) for a in range(len(R1)) for b in range(len(R2)) for c in range(len(R3))
                  for d in range(len(R4)) for e in range(len(R5)) for f in range(len(R6))  if a != b and a != c and a != d
                and a != e and a != f and b != c and b != d and b != e and b != f and c != d and c != e and c != f and d != e and d != f and e != f]

                #ESTA CELDA LA VOY A BORRAR PORUQE NO LA USE, PERO LA DEJO SI TE SIRVE PARA ALGO EN OTRA PREGUNTA, LO QUE HACE ES SACAR TODAS LAS PERMUTACIONES POSIBLES



In [18]:
#ahora usaremos gurobi para optimizar el problema:
from gurobipy import*

D = D = np.array([[4,6,5,9,17,4],[8,12,3,7,11,5],[6,8,2,10,15,7],[5,9,4,8,13,6],
 [7,11,3,12,8,10],[10,5,6,4,9,7]])


n = D.shape[0] # numeros de elemento de elementos de objetos que coincide con el numero de robots

model = Model("Robots_Trabajadores")
model.setParam('OutputFlag',0)

#definamos las varaibles del modelos

#como vamos a trabajar con variables binarias, se tiene que x_{i,j} toma valores {0,1} segun las condiciones colocadas en el informe.


x = model.addVars(6, 6, lb=0, ub=1, vtype=GRB.CONTINUOUS, name="x")

tanh_D= np.tanh(D-5)

model.update()
#Definimos la función objetivo:

model.setObjective(gp.quicksum(C[i, j] * x[i, j] for i in range(6) for j in range(6)), GRB.MINIMIZE)


#restricciones:

# Cada objeto debe asignarse totalmente
for i in range(6):
    model.addConstr(gp.quicksum(x[i, j] for j in range(6)) == 1)

# Cada caja debe recibir exactamente un objeto
for j in range(6):
    model.addConstr(gp.quicksum(x[i, j] for i in range(6)) == 1)

model.optimize()

costo_final = 0
for i in range(n):
    for j in range(n):
        if x[i, j].X > 0.5:
            costo = 50000 * tanh_D[i, j] + 50000
            costo_final += costo
            print(f"Robot {i} asignado a tarea {j} con costo {costo:.2f}")

print(f"\nCosto total de la asignación: {costo_final:.2f}")

Robot 0 asignado a tarea 0 con costo 11920.29
Robot 1 asignado a tarea 5 con costo 50000.00
Robot 2 asignado a tarea 2 con costo 247.26
Robot 3 asignado a tarea 1 con costo 99966.46
Robot 4 asignado a tarea 4 con costo 99752.74
Robot 5 asignado a tarea 3 con costo 11920.29

Costo total de la asignación: 273807.05
